In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [12]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, 
                                names=['tuned_dlm_rm3','orignal_dlm_rm3'], save_mode = "overwrite", 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"],  save_dir = "/workspace/src/runs_dlm_rm3"))
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20original_tuned_dlm_rm3', '20orignal_dlm_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21original_tuned_dlm_rm3', '21orignal_dlm_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/runs_dlm_rm3/original_tuned_dlm_rm3.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.559156
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7ff86d3c5bd0> fb_terms=6', '<pyterrier.rewrite.RM3 object at 0x7ff86d3c5bd0> fb_docs=4', '<pyterrier.rewrite.RM3 object at 0x7ff86d3c5bd0> fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.461935
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7ff86d3c5bd0> fb_terms=4', '<pyterrier.rewrite.RM3 object at 0x7ff86d3c5bd0> fb_docs=8', '<pyterrier.rewrite.RM3 object at 0x7ff86d3c5bd0> fb_lambda=0.2']
              name  ndcg_cut_5  ndcg_cut_25     bpref
0    tuned_dlm_rm3    0.482822     0.336799  0.492538
1  orignal_dlm_rm3    0.552341     0.401122  0.513215
                       name  ndcg_cut_5  ndcg_cut_25     bpref
0  20original_tuned_dlm_rm3    0.425132     0.269420  0.411027
1         20orignal_dlm_rm3    0.463348     0.282512  0.425434
                       name  ndcg_cut_5  ndcg_cu

In [13]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3', folders[i] + '_orignal_dlm_rm3'], save_mode = "overwrite", eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"],  save_dir = "/workspace/src/runs_dlm_rm3"))
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm_rm3', '20' + folders[i] + '_dlm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm_rm3', '21'+ folders[i] + '_dlm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/runs_dlm_rm3/{folders[i]}_tuned_dlm_rm3.csv")
            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.518446
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7ff7f13522d0> fb_terms=12', '<pyterrier.rewrite.RM3 object at 0x7ff7f13522d0> fb_docs=6', '<pyterrier.rewrite.RM3 object at 0x7ff7f13522d0> fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.384881
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7ff7f13522d0> fb_terms=8', '<pyterrier.rewrite.RM3 object at 0x7ff7f13522d0> fb_docs=6', '<pyterrier.rewrite.RM3 object at 0x7ff7f13522d0> fb_lambda=0.2']
                          name  ndcg_cut_5  ndcg_cut_25     bpref
0    conclusions_tuned_dlm_rm3    0.392160     0.303352  0.581750
1  conclusions_orignal_dlm_rm3    0.489808     0.394094  0.599061
                         name  ndcg_cut_5  ndcg_cut_25     bpref
0  20conclusionstuned_dlm_rm3    0.331383     0.236626  0.453295
1       20conclusions_dlm_rm3    0.372117     0.261202  0.469089
       